In [4]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime, timedelta

class WorkHoursApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Calcul des Heures Travaillées")

        # Configuration des colonnes
        self.tree = ttk.Treeview(root, columns=("date", "start", "end", "worked"), show="headings")
        self.tree.heading("date", text="Date")
        self.tree.heading("start", text="Début")
        self.tree.heading("end", text="Fin")
        self.tree.heading("worked", text="Heures Travaillées")
        self.tree.pack(fill=tk.BOTH, expand=True)

        # Événement double-clic pour modification
        self.tree.bind("<Double-1>", self.edit_entry)

        # Champs de saisie
        input_frame = tk.Frame(root)
        input_frame.pack(pady=10)

        tk.Label(input_frame, text="Jour:").grid(row=0, column=0, padx=5, pady=5)
        self.day_combo = ttk.Combobox(input_frame, values=[f"{i:02}" for i in range(1, 32)], width=5, state="readonly")
        self.day_combo.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(input_frame, text="Mois:").grid(row=0, column=2, padx=5, pady=5)
        self.month_combo = ttk.Combobox(input_frame, values=[f"{i:02}" for i in range(1, 13)], width=5, state="readonly")
        self.month_combo.grid(row=0, column=3, padx=5, pady=5)

        current_year = datetime.now().year
        tk.Label(input_frame, text="Année:").grid(row=0, column=4, padx=5, pady=5)
        self.year_combo = ttk.Combobox(input_frame, values=[ str(current_year - 1),str(current_year), str(current_year + 1)], width=5, state="readonly")
        self.year_combo.grid(row=0, column=5, padx=5, pady=5)

        tk.Label(input_frame, text="Heure Début:").grid(row=1, column=0, padx=5, pady=5)
        self.start_hour_combo = ttk.Combobox(input_frame, values=[f"{i:02}" for i in range(0, 24)], width=5, state="readonly")
        self.start_hour_combo.grid(row=1, column=1, padx=5, pady=5)

        tk.Label(input_frame, text="Minute Début:").grid(row=1, column=2, padx=5, pady=5)
        self.start_minute_combo = ttk.Combobox(input_frame, values=[f"{i:02}" for i in range(0, 60)], width=5, state="readonly")
        self.start_minute_combo.grid(row=1, column=3, padx=5, pady=5)

        tk.Label(input_frame, text="Heure Fin:").grid(row=2, column=0, padx=5, pady=5)
        self.end_hour_combo = ttk.Combobox(input_frame, values=[f"{i:02}" for i in range(0, 24)], width=5, state="readonly")
        self.end_hour_combo.grid(row=2, column=1, padx=5, pady=5)

        tk.Label(input_frame, text="Minute Fin:").grid(row=2, column=2, padx=5, pady=5)
        self.end_minute_combo = ttk.Combobox(input_frame, values=[f"{i:02}" for i in range(0, 60)], width=5, state="readonly")
        self.end_minute_combo.grid(row=2, column=3, padx=5, pady=5)

        # Boutons
        button_frame = tk.Frame(root)
        button_frame.pack(pady=10)

        self.add_button = tk.Button(button_frame, text="Ajouter", command=self.add_entry)
        self.add_button.grid(row=0, column=0, padx=10)

        self.total_button = tk.Button(button_frame, text="Total Heures", command=self.calculate_total)
        self.total_button.grid(row=0, column=1, padx=10)

        self.delete_button = tk.Button(button_frame, text="Supprimer", command=self.delete_selected)
        self.delete_button.grid(row=0, column=2, padx=10)

        self.total_label = tk.Label(root, text="Total Heures Travaillées: 0 h 0 min")
        self.total_label.pack(pady=10)

        self.save_button = None

    def add_entry(self):
        try:
            day = self.day_combo.get()
            month = self.month_combo.get()
            year = self.year_combo.get()
            start_hour = self.start_hour_combo.get()
            start_minute = self.start_minute_combo.get()
            end_hour = self.end_hour_combo.get()
            end_minute = self.end_minute_combo.get()

            if not all([day, month, year, start_hour, start_minute, end_hour, end_minute]):
                raise ValueError("Tous les champs doivent être remplis.")

            date = f"{day}/{month}/{year}"
            start_time = f"{start_hour}:{start_minute}"
            end_time = f"{end_hour}:{end_minute}"

            start = datetime.strptime(f"{date} {start_time}", "%d/%m/%Y %H:%M")
            end = datetime.strptime(f"{date} {end_time}", "%d/%m/%Y %H:%M")

            # Gestion des heures de fin après minuit
            if end < start:
                end += timedelta(days=1)

            worked_time = end - start
            worked_hours = worked_time.total_seconds() // 3600
            worked_minutes = (worked_time.total_seconds() % 3600) // 60

            self.tree.insert("", "end", values=(date, start_time, end_time, f"{int(worked_hours)} h {int(worked_minutes)} min"))

            self.update_total_minutes()

            # Réinitialiser les comboboxes
            for combo in [self.day_combo, self.month_combo, self.year_combo, self.start_hour_combo, self.start_minute_combo, self.end_hour_combo, self.end_minute_combo]:
                combo.set("")

        except ValueError as e:
            messagebox.showerror("Erreur", str(e))

    def update_total_minutes(self):
        self.total_minutes = 0
        for child in self.tree.get_children():
            item = self.tree.item(child)
            worked_time_str = item['values'][3]
            parts = worked_time_str.split(' h ')
            hours = int(parts[0])
            minutes = int(parts[1].split(' min')[0])
            self.total_minutes += hours * 60 + minutes

    def calculate_total(self):
        self.update_total_minutes()
        total_hours = self.total_minutes // 60
        total_minutes = self.total_minutes % 60
        self.total_label.config(text=f"Total Heures Travaillées: {int(total_hours)} h {int(total_minutes)} min")

    def delete_selected(self):
        selected_item = self.tree.selection()
        if selected_item:
            for item in selected_item:
                self.tree.delete(item)
            self.update_total_minutes()
            self.calculate_total()
        else:
            messagebox.showwarning("Avertissement", "Veuillez sélectionner une ligne à supprimer.")

    def edit_entry(self, event):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            values = item['values']

            day, month, year = values[0].split('/')
            start_hour, start_minute = values[1].split(':')
            end_hour, end_minute = values[2].split(':')

            self.day_combo.set(day)
            self.month_combo.set(month)
            self.year_combo.set(year)
            self.start_hour_combo.set(start_hour)
            self.start_minute_combo.set(start_minute)
            self.end_hour_combo.set(end_hour)
            self.end_minute_combo.set(end_minute)

            if self.save_button is not None:
                self.save_button.destroy()

            def save_changes():
                try:
                    self.add_entry()
                    self.tree.delete(selected_item)
                    self.save_button.destroy()
                    self.save_button = None
                except ValueError as e:
                    messagebox.showerror("Erreur", str(e))

            self.save_button = tk.Button(self.root, text="Sauvegarder", command=save_changes)
            self.save_button.pack(pady=5)

if __name__ == "__main__":
    root = tk.Tk()
    app = WorkHoursApp(root)
    root.mainloop()


In [11]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from datetime import datetime, timedelta
from tkcalendar import DateEntry
import locale

# Configurer la locale française
try:
    locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # Pour Linux/Mac
except:
    try:
        locale.setlocale(locale.LC_TIME, 'fra_fra')  # Pour Windows
    except:
        try:
            locale.setlocale(locale.LC_TIME, 'fr_FR')  # Alternative
        except:
            pass  # Si aucune locale française n'est disponible
import customtkinter as ctk
import os
from PIL import ImageGrab
import pandas as pd
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4, landscape
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

class WorkHoursApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Calcul des Heures Travaillées")
        self.root.geometry("1100x700")
        self.root.configure(bg="#f0f0f0")
        
        # Variables
        self.is_night_shift = tk.BooleanVar(value=False)
        self.hourly_rate = tk.DoubleVar(value=0.0)
        self.current_id = 0
        self.entries = []
        self.editing_id = None
        
        # Styles
        self.style = ttk.Style()
        self.style.theme_use("clam")
        self.style.configure("Treeview", background="#f5f5f5", fieldbackground="#f5f5f5", foreground="#333333")
        self.style.configure("Treeview.Heading", font=('Arial', 10, 'bold'), background="#4a7a8c", foreground="white")
        self.style.map('Treeview', background=[('selected', '#4a7a8c')])
        
        # Frame principal
        main_frame = tk.Frame(root, bg="#f0f0f0")
        main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        # Titre
        title_label = tk.Label(main_frame, text="GESTION DES HEURES TRAVAILLÉES", font=("Arial", 18, "bold"), 
                              bg="#f0f0f0", fg="#4a7a8c")
        title_label.pack(pady=(0, 20))
        
        # Section 1: Configuration
        config_frame = tk.LabelFrame(main_frame, text="Configuration", font=("Arial", 12, "bold"), 
                                    bg="#f0f0f0", fg="#4a7a8c", padx=10, pady=10)
        config_frame.pack(fill=tk.X, pady=10)
        
        # Type de shift
        shift_frame = tk.Frame(config_frame, bg="#f0f0f0")
        shift_frame.pack(fill=tk.X, pady=5)
        
        shift_label = tk.Label(shift_frame, text="Type de travail:", font=("Arial", 10), bg="#f0f0f0")
        shift_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        
        day_shift_rb = tk.Radiobutton(shift_frame, text="Journée (même jour)", variable=self.is_night_shift, 
                                     value=False, command=self.update_date_fields, bg="#f0f0f0", font=("Arial", 10))
        day_shift_rb.grid(row=0, column=1, padx=5, pady=5)
        
        night_shift_rb = tk.Radiobutton(shift_frame, text="Nuit (jour suivant)", variable=self.is_night_shift, 
                                       value=True, command=self.update_date_fields, bg="#f0f0f0", font=("Arial", 10))
        night_shift_rb.grid(row=0, column=2, padx=5, pady=5)
        
        # Taux horaire
        rate_frame = tk.Frame(config_frame, bg="#f0f0f0")
        rate_frame.pack(fill=tk.X, pady=5)
        
        rate_label = tk.Label(rate_frame, text="Taux horaire (€):", font=("Arial", 10), bg="#f0f0f0")
        rate_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        
        rate_entry = tk.Entry(rate_frame, textvariable=self.hourly_rate, width=10, font=("Arial", 10))
        rate_entry.grid(row=0, column=1, padx=5, pady=5, sticky="w")
        
        # Section 2: Saisie
        input_frame = tk.LabelFrame(main_frame, text="Saisie des Heures", font=("Arial", 12, "bold"), 
                                   bg="#f0f0f0", fg="#4a7a8c", padx=10, pady=10)
        input_frame.pack(fill=tk.X, pady=10)
        
        # Dates
        date_frame = tk.Frame(input_frame, bg="#f0f0f0")
        date_frame.pack(fill=tk.X, pady=5)
        
        start_date_label = tk.Label(date_frame, text="Date de début:", font=("Arial", 10), bg="#f0f0f0")
        start_date_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        
        self.start_date_entry = DateEntry(date_frame, width=12, background='#4a7a8c',
                                        foreground='white', borderwidth=2, font=("Arial", 10),
                                        locale='fr_FR', date_pattern='dd/MM/yyyy')
        self.start_date_entry.grid(row=0, column=1, padx=5, pady=5)
        
        end_date_label = tk.Label(date_frame, text="Date de fin:", font=("Arial", 10), bg="#f0f0f0")
        end_date_label.grid(row=0, column=2, padx=5, pady=5, sticky="w")
        
        self.end_date_entry = DateEntry(date_frame, width=12, background='#4a7a8c',
                                      foreground='white', borderwidth=2, font=("Arial", 10), state="readonly",
                                      locale='fr_FR', date_pattern='dd/MM/yyyy')
        self.end_date_entry.grid(row=0, column=3, padx=5, pady=5)
        
        # Heures
        time_frame = tk.Frame(input_frame, bg="#f0f0f0")
        time_frame.pack(fill=tk.X, pady=5)
        
        start_time_label = tk.Label(time_frame, text="Heure de début:", font=("Arial", 10), bg="#f0f0f0")
        start_time_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        
        self.start_hour_combo = ttk.Combobox(time_frame, values=[f"{i:02}" for i in range(0, 24)], 
                                           width=5, font=("Arial", 10), state="readonly")
        self.start_hour_combo.grid(row=0, column=1, padx=5, pady=5)
        
        tk.Label(time_frame, text=":", bg="#f0f0f0").grid(row=0, column=2)
        
        self.start_minute_combo = ttk.Combobox(time_frame, values=[f"{i:02}" for i in range(0, 60, 5)], 
                                             width=5, font=("Arial", 10), state="readonly")
        self.start_minute_combo.grid(row=0, column=3, padx=5, pady=5)
        
        end_time_label = tk.Label(time_frame, text="Heure de fin:", font=("Arial", 10), bg="#f0f0f0")
        end_time_label.grid(row=0, column=4, padx=(20, 5), pady=5, sticky="w")
        
        self.end_hour_combo = ttk.Combobox(time_frame, values=[f"{i:02}" for i in range(0, 24)], 
                                         width=5, font=("Arial", 10), state="readonly")
        self.end_hour_combo.grid(row=0, column=5, padx=5, pady=5)
        
        tk.Label(time_frame, text=":", bg="#f0f0f0").grid(row=0, column=6)
        
        self.end_minute_combo = ttk.Combobox(time_frame, values=[f"{i:02}" for i in range(0, 60, 5)], 
                                           width=5, font=("Arial", 10), state="readonly")
        self.end_minute_combo.grid(row=0, column=7, padx=5, pady=5)
        
        # Boutons d'action
        button_frame = tk.Frame(input_frame, bg="#f0f0f0")
        button_frame.pack(fill=tk.X, pady=10)
        
        self.add_button = tk.Button(button_frame, text="Ajouter", command=self.add_entry,
                                  bg="#4a7a8c", fg="white", font=("Arial", 10, "bold"),
                                  width=12, relief=tk.RAISED, borderwidth=2)
        self.add_button.pack(side=tk.LEFT, padx=5)
        
        self.update_button = tk.Button(button_frame, text="Mettre à jour", command=self.update_entry,
                                     bg="#4a7a8c", fg="white", font=("Arial", 10, "bold"),
                                     width=12, relief=tk.RAISED, borderwidth=2, state=tk.DISABLED)
        self.update_button.pack(side=tk.LEFT, padx=5)
        
        self.cancel_button = tk.Button(button_frame, text="Annuler", command=self.cancel_edit,
                                     bg="#d9534f", fg="white", font=("Arial", 10, "bold"),
                                     width=12, relief=tk.RAISED, borderwidth=2, state=tk.DISABLED)
        self.cancel_button.pack(side=tk.LEFT, padx=5)
        
        # Section 3: Tableau des données
        tree_frame = tk.LabelFrame(main_frame, text="Registre des Heures", font=("Arial", 12, "bold"), 
                                  bg="#f0f0f0", fg="#4a7a8c", padx=10, pady=10)
        tree_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # Tableau
        columns = ("id", "date_debut", "date_fin", "heure_debut", "heure_fin", "duree", "taux", "montant")
        self.tree = ttk.Treeview(tree_frame, columns=columns, show="headings", height=10)
        
        # Définition des colonnes
        self.tree.heading("id", text="#")
        self.tree.heading("date_debut", text="Date Début")
        self.tree.heading("date_fin", text="Date Fin")
        self.tree.heading("heure_debut", text="Heure Début")
        self.tree.heading("heure_fin", text="Heure Fin")
        self.tree.heading("duree", text="Durée")
        self.tree.heading("taux", text="Taux (€/h)")
        self.tree.heading("montant", text="Montant (€)")
        
        # Largeur des colonnes
        self.tree.column("id", width=30, anchor="center")
        self.tree.column("date_debut", width=100, anchor="center")
        self.tree.column("date_fin", width=100, anchor="center")
        self.tree.column("heure_debut", width=100, anchor="center")
        self.tree.column("heure_fin", width=100, anchor="center")
        self.tree.column("duree", width=100, anchor="center")
        self.tree.column("taux", width=100, anchor="center")
        self.tree.column("montant", width=100, anchor="center")
        
        # Scrollbar
        scrollbar = ttk.Scrollbar(tree_frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.tree.pack(fill=tk.BOTH, expand=True)
        
        # Événements
        self.tree.bind("<Double-1>", self.on_tree_double_click)
        self.tree.bind("<ButtonRelease-1>", self.on_tree_select)
        
        # Boutons pour le tableau
        tree_buttons_frame = tk.Frame(tree_frame, bg="#f0f0f0")
        tree_buttons_frame.pack(fill=tk.X, pady=5)
        
        self.delete_button = tk.Button(tree_buttons_frame, text="Supprimer", command=self.delete_selected,
                                     bg="#d9534f", fg="white", font=("Arial", 10, "bold"),
                                     width=12, relief=tk.RAISED, borderwidth=2)
        self.delete_button.pack(side=tk.LEFT, padx=5)
        
        # Bouton d'exportation
        self.export_button = tk.Button(tree_buttons_frame, text="Exporter", command=self.show_export_options,
                                     bg="#28a745", fg="white", font=("Arial", 10, "bold"),
                                     width=12, relief=tk.RAISED, borderwidth=2)
        self.export_button.pack(side=tk.LEFT, padx=5)
        
        # Section 4: Résumé
        summary_frame = tk.LabelFrame(main_frame, text="Résumé", font=("Arial", 12, "bold"), 
                                     bg="#f0f0f0", fg="#4a7a8c", padx=10, pady=10)
        summary_frame.pack(fill=tk.X, pady=10)
        
        summary_inner_frame = tk.Frame(summary_frame, bg="#f0f0f0")
        summary_inner_frame.pack(fill=tk.X, pady=5)
        
        # Filtres de date
        filter_frame = tk.Frame(summary_inner_frame, bg="#f0f0f0")
        filter_frame.grid(row=0, column=0, padx=10, pady=5, sticky="w")
        
        from_date_label = tk.Label(filter_frame, text="Du:", font=("Arial", 10), bg="#f0f0f0")
        from_date_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        
        self.from_date_entry = DateEntry(filter_frame, width=12, background='#4a7a8c',
                                       foreground='white', borderwidth=2, font=("Arial", 10),
                                       locale='fr_FR', date_pattern='dd/MM/yyyy')
        self.from_date_entry.grid(row=0, column=1, padx=5, pady=5)
        
        to_date_label = tk.Label(filter_frame, text="Au:", font=("Arial", 10), bg="#f0f0f0")
        to_date_label.grid(row=0, column=2, padx=5, pady=5, sticky="w")
        
        self.to_date_entry = DateEntry(filter_frame, width=12, background='#4a7a8c',
                                     foreground='white', borderwidth=2, font=("Arial", 10),
                                     locale='fr_FR', date_pattern='dd/MM/yyyy')
        self.to_date_entry.grid(row=0, column=3, padx=5, pady=5)
        
        filter_button = tk.Button(filter_frame, text="Filtrer", command=self.filter_entries,
                                bg="#4a7a8c", fg="white", font=("Arial", 10), width=8)
        filter_button.grid(row=0, column=4, padx=5, pady=5)
        
        reset_button = tk.Button(filter_frame, text="Réinitialiser", command=self.reset_filter,
                               bg="#868e96", fg="white", font=("Arial", 10), width=10)
        reset_button.grid(row=0, column=5, padx=5, pady=5)
        
        # Totaux
        totals_frame = tk.Frame(summary_inner_frame, bg="#f0f0f0")
        totals_frame.grid(row=1, column=0, padx=10, pady=5, sticky="w")
        
        self.total_hours_label = tk.Label(totals_frame, text="Heures totales: 0h 0m", 
                                        font=("Arial", 12, "bold"), bg="#f0f0f0", fg="#4a7a8c")
        self.total_hours_label.grid(row=0, column=0, padx=10, pady=5, sticky="w")
        
        self.total_amount_label = tk.Label(totals_frame, text="Montant total: 0.00 €", 
                                         font=("Arial", 12, "bold"), bg="#f0f0f0", fg="#4a7a8c")
        self.total_amount_label.grid(row=0, column=1, padx=10, pady=5, sticky="w")
        
        # Initialisation
        self.update_date_fields()
        self.set_default_values()
    
    def set_default_values(self):
        """Définit les valeurs par défaut pour les champs"""
        now = datetime.now()
        
        # Initialisation des dates
        self.start_date_entry.set_date(now)
        self.end_date_entry.set_date(now)
        self.from_date_entry.set_date(now.replace(day=1))  # Premier jour du mois
        self.to_date_entry.set_date(now)
        
        # Initialisation des heures
        self.start_hour_combo.set("08")
        self.start_minute_combo.set("00")
        self.end_hour_combo.set("17")
        self.end_minute_combo.set("00")
        
        # Taux horaire par défaut
        self.hourly_rate.set(10.0)
    
    def update_date_fields(self):
        """Met à jour les champs de date selon le type de shift"""
        if self.is_night_shift.get():
            # Pour les shifts de nuit, la date de fin est automatiquement le jour suivant
            self.end_date_entry.config(state="readonly")
            start_date = self.start_date_entry.get_date()
            end_date = start_date + timedelta(days=1)
            self.end_date_entry.set_date(end_date)
        else:
            # Pour les shifts de jour, les deux dates sont identiques
            self.end_date_entry.config(state="readonly")
            self.end_date_entry.set_date(self.start_date_entry.get_date())
        
        # Mettre à jour la date de fin quand la date de début change
        self.start_date_entry.bind("<<DateEntrySelected>>", lambda e: self.update_end_date())
    
    def update_end_date(self):
        """Met à jour la date de fin lorsque la date de début change"""
        if self.is_night_shift.get():
            start_date = self.start_date_entry.get_date()
            end_date = start_date + timedelta(days=1)
        else:
            start_date = self.start_date_entry.get_date()
            end_date = start_date
        
        self.end_date_entry.set_date(end_date)
    
    def add_entry(self):
        """Ajoute une nouvelle entrée dans le tableau"""
        try:
            # Récupération des valeurs
            start_date = self.start_date_entry.get_date()
            end_date = self.end_date_entry.get_date()
            start_hour = self.start_hour_combo.get()
            start_minute = self.start_minute_combo.get()
            end_hour = self.end_hour_combo.get()
            end_minute = self.end_minute_combo.get()
            hourly_rate = self.hourly_rate.get()
            
            # Validation
            if not all([start_hour, start_minute, end_hour, end_minute]):
                messagebox.showerror("Erreur", "Veuillez remplir tous les champs d'heure.")
                return
            
            if hourly_rate <= 0:
                messagebox.showerror("Erreur", "Le taux horaire doit être supérieur à 0.")
                return
            
            # Création des objets datetime
            start_datetime = datetime.combine(start_date, datetime.strptime(f"{start_hour}:{start_minute}", "%H:%M").time())
            end_datetime = datetime.combine(end_date, datetime.strptime(f"{end_hour}:{end_minute}", "%H:%M").time())
            
            # Vérification que l'heure de fin est après l'heure de début
            if end_datetime <= start_datetime:
                messagebox.showerror("Erreur", "L'heure de fin doit être après l'heure de début.")
                return
            
            # Calcul de la durée
            duration = end_datetime - start_datetime
            hours = duration.total_seconds() // 3600
            minutes = (duration.total_seconds() % 3600) // 60
            duration_str = f"{int(hours)}h {int(minutes)}m"
            
            # Calcul du montant
            amount = (hours + minutes/60) * hourly_rate
            
            # Formatage des dates pour l'affichage
            start_date_str = start_date.strftime("%d/%m/%Y")
            end_date_str = end_date.strftime("%d/%m/%Y")
            start_time_str = f"{start_hour}:{start_minute}"
            end_time_str = f"{end_hour}:{end_minute}"
            
            # Incrémentation de l'ID
            self.current_id += 1
            
            # Ajout dans le tableau
            entry_id = self.current_id
            entry = (entry_id, start_date_str, end_date_str, start_time_str, end_time_str, 
                    duration_str, f"{hourly_rate:.2f}", f"{amount:.2f}")
            
            self.tree.insert("", "end", values=entry)
            
            # Stocker les données pour les filtres ultérieurs
            entry_data = {
                "id": entry_id,
                "start_date": start_date,
                "end_date": end_date,
                "start_time": start_time_str,
                "end_time": end_time_str,
                "duration": duration,
                "duration_str": duration_str,
                "hourly_rate": hourly_rate,
                "amount": amount
            }
            self.entries.append(entry_data)
            
            # Mettre à jour les totaux
            self.update_totals()
            
        except Exception as e:
            messagebox.showerror("Erreur", str(e))
    
    def update_entry(self):
        """Met à jour une entrée existante"""
        if self.editing_id is None:
            return
        
        # Supprimer l'ancienne entrée
        for item in self.tree.get_children():
            if int(self.tree.item(item)["values"][0]) == self.editing_id:
                self.tree.delete(item)
                # Supprimer des données stockées
                self.entries = [entry for entry in self.entries if entry["id"] != self.editing_id]
                break
        
        # Ajouter la nouvelle entrée avec le même ID
        old_id = self.current_id
        self.current_id = self.editing_id - 1  # -1 car add_entry incrémente
        self.add_entry()
        self.current_id = old_id
        
        # Réinitialiser l'état d'édition
        self.editing_id = None
        self.update_button.config(state=tk.DISABLED)
        self.cancel_button.config(state=tk.DISABLED)
        self.add_button.config(state=tk.NORMAL)
    
    def cancel_edit(self):
        """Annule l'édition en cours"""
        self.editing_id = None
        self.update_button.config(state=tk.DISABLED)
        self.cancel_button.config(state=tk.DISABLED)
        self.add_button.config(state=tk.NORMAL)
        self.set_default_values()
    
    def on_tree_double_click(self, event):
        """Gère le double-clic sur une ligne du tableau"""
        item = self.tree.identify_row(event.y)
        if not item:
            return
            
        # Récupérer les valeurs
        values = self.tree.item(item)["values"]
        entry_id = int(values[0])
        
        # Remplir les champs avec les valeurs sélectionnées
        start_date = datetime.strptime(values[1], "%d/%m/%Y").date()
        end_date = datetime.strptime(values[2], "%d/%m/%Y").date()
        
        # Détermine si c'est un shift de nuit
        is_night = start_date != end_date
        self.is_night_shift.set(is_night)
        
        # Mise à jour des champs de date
        self.start_date_entry.set_date(start_date)
        self.end_date_entry.set_date(end_date)
        
        # Mise à jour des champs d'heure
        start_hour, start_minute = values[3].split(":")
        end_hour, end_minute = values[4].split(":")
        
        self.start_hour_combo.set(start_hour)
        self.start_minute_combo.set(start_minute)
        self.end_hour_combo.set(end_hour)
        self.end_minute_combo.set(end_minute)
        
        # Mise à jour du taux horaire
        self.hourly_rate.set(float(values[6]))
        
        # Activer le mode édition
        self.editing_id = entry_id
        self.update_button.config(state=tk.NORMAL)
        self.cancel_button.config(state=tk.NORMAL)
        self.add_button.config(state=tk.DISABLED)
    
    def on_tree_select(self, event):
        """Gère la sélection d'une ligne du tableau"""
        pass
    
    def delete_selected(self):
        """Supprime les entrées sélectionnées"""
        selected_items = self.tree.selection()
        if not selected_items:
            messagebox.showwarning("Avertissement", "Veuillez sélectionner une ligne à supprimer.")
            return
            
        if messagebox.askyesno("Confirmation", "Êtes-vous sûr de vouloir supprimer les entrées sélectionnées?"):
            for item in selected_items:
                entry_id = int(self.tree.item(item)["values"][0])
                self.tree.delete(item)
                # Supprimer des données stockées
                self.entries = [entry for entry in self.entries if entry["id"] != entry_id]
            
            # Mettre à jour les totaux
            self.update_totals()
    
    def filter_entries(self):
        """Filtre les entrées selon la plage de dates"""
        from_date = self.from_date_entry.get_date()
        to_date = self.to_date_entry.get_date()
        
        # Effacer le tableau
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Filtrer et réinsérer
        filtered_entries = []
        for entry in self.entries:
            start_date = entry["start_date"]
            if from_date <= start_date <= to_date:
                filtered_entries.append(entry)
                # Recréer la ligne dans le tableau
                values = (
                    entry["id"],
                    start_date.strftime("%d/%m/%Y"),
                    entry["end_date"].strftime("%d/%m/%Y"),
                    entry["start_time"],
                    entry["end_time"],
                    entry["duration_str"],
                    f"{entry['hourly_rate']:.2f}",
                    f"{entry['amount']:.2f}"
                )
                self.tree.insert("", "end", values=values)
        
        # Mettre à jour les totaux pour les entrées filtrées
        self.update_totals(filtered_entries)
    
    def reset_filter(self):
        """Réinitialise le filtre et affiche toutes les entrées"""
        # Effacer le tableau
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Réinsérer toutes les entrées
        for entry in self.entries:
            values = (
                entry["id"],
                entry["start_date"].strftime("%d/%m/%Y"),
                entry["end_date"].strftime("%d/%m/%Y"),
                entry["start_time"],
                entry["end_time"],
                entry["duration_str"],
                f"{entry['hourly_rate']:.2f}",
                f"{entry['amount']:.2f}"
            )
            self.tree.insert("", "end", values=values)
        
        # Mettre à jour les totaux
        self.update_totals()
    
    def update_totals(self, entries_to_sum=None):
        """Met à jour les totaux affichés"""
        if entries_to_sum is None:
            entries_to_sum = self.entries
        
        total_hours = 0
        total_amount = 0
        
        for entry in entries_to_sum:
            total_hours += entry["duration"].total_seconds() / 3600
            total_amount += entry["amount"]
        
        hours = int(total_hours)
        minutes = int((total_hours - hours) * 60)
        
        self.total_hours_label.config(text=f"Heures totales: {hours}h {minutes}m")
        self.total_amount_label.config(text=f"Montant total: {total_amount:.2f} €")

    def show_export_options(self):
        """Affiche une fenêtre de dialogue pour choisir le format d'exportation"""
        export_window = tk.Toplevel(self.root)
        export_window.title("Exporter les données")
        export_window.geometry("350x300")
        export_window.configure(bg="#f0f0f0")
        export_window.resizable(False, False)
        
        # Centrer la fenêtre
        export_window.geometry(f"+{self.root.winfo_x() + 400}+{self.root.winfo_y() + 200}")
        
        # Titre
        tk.Label(export_window, text="Choisissez un format d'exportation", 
               font=("Arial", 12, "bold"), bg="#f0f0f0", fg="#4a7a8c").pack(pady=15)
        
        # Boutons d'exportation
        button_frame = tk.Frame(export_window, bg="#f0f0f0")
        button_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        excel_button = tk.Button(button_frame, text="Excel (.xlsx)", command=lambda: self.export_data("excel"),
                              bg="#217346", fg="white", font=("Arial", 10, "bold"),
                              width=15, height=2, relief=tk.RAISED)
        excel_button.pack(fill=tk.X, pady=5)
        
        pdf_button = tk.Button(button_frame, text="PDF (.pdf)", command=lambda: self.export_data("pdf"),
                             bg="#F40F02", fg="white", font=("Arial", 10, "bold"),
                             width=15, height=2, relief=tk.RAISED)
        pdf_button.pack(fill=tk.X, pady=5)
        
        image_button = tk.Button(button_frame, text="Image (.png)", command=lambda: self.export_data("png"),
                               bg="#FFD700", fg="black", font=("Arial", 10, "bold"),
                               width=15, height=2, relief=tk.RAISED)
        image_button.pack(fill=tk.X, pady=5)
        
        # Annuler
        cancel_button = tk.Button(export_window, text="Annuler", command=export_window.destroy,
                                bg="#d9534f", fg="white", font=("Arial", 10))
        cancel_button.pack(pady=10)
        
        # Rendre la fenêtre modale
        export_window.transient(self.root)
        export_window.grab_set()
        self.root.wait_window(export_window)
    
    def export_data(self, format_type):
        """Exporte les données selon le format choisi"""
        if not self.tree.get_children():
            messagebox.showwarning("Avertissement", "Aucune donnée à exporter.")
            return
        
        # Obtenir la date actuelle pour le nom de fichier
        current_date = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        try:
            if format_type == "excel":
                # Demander où sauvegarder le fichier
                file_path = filedialog.asksaveasfilename(
                    defaultextension=".xlsx",
                    filetypes=[("Excel files", "*.xlsx")],
                    initialfile=f"Heures_Travaillees_{current_date}.xlsx"
                )
                
                if not file_path:
                    return
                
                # Créer un DataFrame avec les données
                data = []
                columns = ["ID", "Date Début", "Date Fin", "Heure Début", "Heure Fin", 
                          "Durée", "Taux (€/h)", "Montant (€)"]
                
                for item_id in self.tree.get_children():
                    values = self.tree.item(item_id)["values"]
                    data.append(values)
                
                df = pd.DataFrame(data, columns=columns)
                
                # Ajouter une ligne pour les totaux
                total_hours = sum(entry["duration"].total_seconds() / 3600 for entry in self.entries)
                hours = int(total_hours)
                minutes = int((total_hours - hours) * 60)
                total_amount = sum(entry["amount"] for entry in self.entries)
                
                totals_row = ["TOTAL", "", "", "", "", f"{hours}h {minutes}m", "", f"{total_amount:.2f}"]
                df.loc[len(df)] = totals_row
                
                # Exporter vers Excel
                df.to_excel(file_path, index=False, sheet_name="Heures Travaillées")
                
                # Ouvrir le fichier
                os.startfile(file_path) if os.name == 'nt' else messagebox.showinfo(
                    "Exportation réussie", f"Le fichier a été enregistré à l'emplacement:\n{file_path}")
            
            elif format_type == "pdf":
                # Demander où sauvegarder le fichier
                file_path = filedialog.asksaveasfilename(
                    defaultextension=".pdf",
                    filetypes=[("PDF files", "*.pdf")],
                    initialfile=f"Heures_Travaillees_{current_date}.pdf"
                )
                
                if not file_path:
                    return
                
                # Créer le document PDF
                doc = SimpleDocTemplate(
                    file_path,
                    pagesize=landscape(A4),
                    title="Rapport des Heures Travaillées"
                )
                
                # Styles
                styles = getSampleStyleSheet()
                title_style = styles['Heading1']
                subtitle_style = styles['Heading2']
                
                # Éléments du document
                elements = []
                
                # Titre
                elements.append(Paragraph("Rapport des Heures Travaillées", title_style))
                elements.append(Spacer(1, 12))
                elements.append(Paragraph(f"Généré le {datetime.now().strftime('%d/%m/%Y à %H:%M')}", styles['Normal']))
                elements.append(Spacer(1, 20))
                
                # Tableau des données
                data = [["ID", "Date Début", "Date Fin", "Heure Début", "Heure Fin", 
                        "Durée", "Taux (€/h)", "Montant (€)"]]
                
                for item_id in self.tree.get_children():
                    values = self.tree.item(item_id)["values"]
                    data.append(values)
                
                # Ajouter une ligne pour les totaux
                total_hours = sum(entry["duration"].total_seconds() / 3600 for entry in self.entries)
                hours = int(total_hours)
                minutes = int((total_hours - hours) * 60)
                total_amount = sum(entry["amount"] for entry in self.entries)
                
                data.append(["TOTAL", "", "", "", "", f"{hours}h {minutes}m", "", f"{total_amount:.2f}"])
                
                # Créer et styliser le tableau
                table = Table(data)
                table.setStyle(TableStyle([
                    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                    ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
                    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                    ('FONTSIZE', (0, 0), (-1, 0), 12),
                    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                    ('BACKGROUND', (0, -1), (-1, -1), colors.lightgrey),
                    ('FONTNAME', (0, -1), (-1, -1), 'Helvetica-Bold'),
                    ('GRID', (0, 0), (-1, -1), 1, colors.black),
                    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                    ('ALIGN', (0, 1), (-1, -1), 'CENTER'),
                ]))
                
                elements.append(table)
                
                # Ajouter les résumés
                elements.append(Spacer(1, 30))
                elements.append(Paragraph("Résumé", subtitle_style))
                elements.append(Spacer(1, 10))
                elements.append(Paragraph(f"Heures totales travaillées: {hours}h {minutes}m", styles['Normal']))
                elements.append(Paragraph(f"Montant total: {total_amount:.2f} €", styles['Normal']))
                
                # Générer le PDF
                doc.build(elements)
                
                # Ouvrir le fichier
                os.startfile(file_path) if os.name == 'nt' else messagebox.showinfo(
                    "Exportation réussie", f"Le fichier a été enregistré à l'emplacement:\n{file_path}")
                
            elif format_type == "png":
                # Demander où sauvegarder le fichier
                file_path = filedialog.asksaveasfilename(
                    defaultextension=".png",
                    filetypes=[("PNG files", "*.png")],
                    initialfile=f"Heures_Travaillees_{current_date}.png"
                )
                
                if not file_path:
                    return
                
                # Préparer la capture d'écran du tableau
                self.root.update()
                
                # Identifier les coordonnées du tableau
                tree_x = self.tree.winfo_rootx()
                tree_y = self.tree.winfo_rooty()
                tree_width = self.tree.winfo_width()
                tree_height = self.tree.winfo_height()
                
                # Prendre une capture d'écran du tableau
                image = ImageGrab.grab((tree_x, tree_y, tree_x + tree_width, tree_y + tree_height))
                image.save(file_path)
                
                # Ouvrir le fichier
                os.startfile(file_path) if os.name == 'nt' else messagebox.showinfo(
                    "Exportation réussie", f"Le fichier a été enregistré à l'emplacement:\n{file_path}")
            
            messagebox.showinfo("Exportation réussie", f"Les données ont été exportées avec succès au format {format_type.upper()}.")
        
        except Exception as e:
            messagebox.showerror("Erreur d'exportation", f"Une erreur s'est produite lors de l'exportation:\n{str(e)}")

def main():
    try:
        import customtkinter as ctk
        ctk.set_appearance_mode("System")
        ctk.set_default_color_theme("blue")
        root = ctk.CTk()
        modern_ui = True
    except ImportError:
        # Fallback en cas de non-disponibilité de customtkinter
        root = tk.Tk()
        modern_ui = False
    
    # Vérification des dépendances
    missing_deps = []
    
    try:
        from tkcalendar import DateEntry
    except ImportError:
        missing_deps.append("tkcalendar")
    
    try:
        import pandas as pd
    except ImportError:
        missing_deps.append("pandas")
    
    try:
        from PIL import ImageGrab
    except ImportError:
        missing_deps.append("pillow")
    
    try:
        from reportlab.platypus import SimpleDocTemplate
    except ImportError:
        missing_deps.append("reportlab")
    
    if missing_deps:
        error_message = "Cette application nécessite les modules suivants:\n\n"
        for dep in missing_deps:
            error_message += f"- {dep}\n"
        error_message += "\nVeuillez les installer avec pip:\n\n"
        error_message += f"pip install {' '.join(missing_deps)}"
        
        messagebox.showerror("Erreur de dépendances", error_message)
        if not modern_ui and "customtkinter" not in missing_deps:
            messagebox.showinfo("Astuce", 
                              "Pour une interface plus moderne, installez aussi customtkinter:\n\npip install customtkinter")
        return
    
    app = WorkHoursApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

KeyboardInterrupt: 